In [2]:
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial import distance_matrix

from sklearn.svm import SVC
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_curve, accuracy_score
from sklearn.model_selection import RandomizedSearchCV

In [3]:
path = "images-photographes"

In [4]:
def load_dataset(dir_sc, images_per_class=None):
    inames = []
    ilabels = []
    cnames = sorted(os.listdir(dir_sc))
    for ilabel, cl in enumerate(cnames):
        dir_cl = os.path.join(dir_sc, cl)
        for iname in os.listdir(dir_cl)[:images_per_class]:
            inames.append(os.path.join(cl, iname))
            ilabels.append(ilabel)
    ilabels = np.array(ilabels)
    return inames, ilabels, cnames


In [5]:
inames, ilabels, class_names = load_dataset(path)

In [6]:
class_names

['daido-moriyama',
 'dorothea-lange',
 'felix-nadar',
 'henri-cartier-bresson',
 'irving-penn',
 'malik-sidibe',
 'marc-riboud',
 'seidou-keita',
 'shomei-tomatsu',
 'walter-evans']

In [7]:
ilabels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9])

# SIFTs COMPUTATION

In [8]:
def dense_sampling(im, s=8):
    w, h = im.shape
    x = np.arange(0, w, s)
    y = np.arange(0, h, s)
    return x, y

In [9]:
def conv_separable(im, h_x, h_y, pad=1):
    h_x = h_x.reshape(1,3)
    h_y = h_y.reshape(3,1)

    im_w, im_h = im.shape
    hx_w, hx_h = h_x.shape
    hy_w, hy_h = h_y.shape

    h_x_t = h_x.transpose()
    h_y_t = h_y.transpose()

    if hx_w != 1:
        raise ValueError()
    if hx_h % 2 != 1:
        raise ValueError()
    if hy_h != 1:
        raise ValueError()
    if hy_w % 2 != 1:
        raise ValueError()
    if hx_h != hy_w:
        raise ValueError()


In [10]:
def auto_padding(im, k=16, s=8):
    w, h = im.shape
    x = np.arange(0, w, s)
    y = np.arange(0, h, s)
    # last region could be smaller
    last_r = im[x[-1]:x[-1]+k, y[-1]:y[-1]+k]
    if last_r.shape == (k, k):
        return im
    dif_w = k - last_r.shape[0]
    dif_h = k - last_r.shape[1]
    n_im = np.zeros((w+dif_w, h+dif_h))
    id_w = dif_w // 2
    id_h = dif_h // 2
    n_im[id_w:id_w+w, id_h:id_h+h] = im
    return n_im

In [11]:
def gaussian_mask(size=16, sigma=0.5):
    sigma *= size
    ax = np.arange(-size // 2 + 1., size // 2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2. * sigma**2))
    return kernel / np.sum(kernel)

In [12]:
def compute_grad(I):

    ha = 1/4 * np.array([1,2,1])
    hb = 1/2 * np.array([1,0,-1])

    Ix = conv_separable(I, hb, ha)
    Iy = conv_separable(I, ha, hb)
    print(Ix)
    print(Iy)
    return Ix, Iy

In [13]:
def compute_grad(I):

    gX = cv.Sobel(I, ddepth=cv.CV_32F, dx=1, dy=0, ksize=3)
    gY = cv.Sobel(I, ddepth=cv.CV_32F, dx=0, dy=1, ksize=3)
    #print(gX)
    #print(gY)
    return gX, gY

In [14]:
def compute_grad_ori(g_x, g_y, g_m, b=8):
    ori = np.zeros((b, 2))
    for i in range(b):
        ori[i,0] = np.cos(2 * np.pi * i / b)
        ori[i,1] = np.sin(2 * np.pi * i / b)
    w, h = g_m.shape
    # TODO: algebraic form
    g_o = np.zeros((w, h))
    for i in range(w):
        for j in range(h):
            if g_m[i,j] > 0:
                v = np.array([g_y[i,j], -g_x[i,j]])
                v = v / np.linalg.norm(v, ord=2)
                prod = np.dot(ori,v)
                g_o[i,j] = np.argmax(prod)
            else:
                g_o[i,j] = -1
    g_o = g_o.astype(int)
    return g_o

In [15]:
def compute_grad_mod_ori(I):


    Ix, Iy = compute_grad(I)

    Gn = np.sqrt(Ix**2 + Iy**2)
    Go = compute_grad_ori(Ix, Iy, Gn, 8)#np.arctan(Iy/Ix)
    return Gn, Go

In [16]:
def compute_histogram(g_n, g_o):
    """
    g_n and g_o are 4x4 matrices that contain the norm, and the discretized orientation.
    Warning: g_o can contain -1, thos1e values should be discarded.
    """
    hist = np.zeros((8))
    for i in range(8):
        hist[i] = g_n[g_o == i].sum()
    return hist

In [17]:
def compute_sift_region(Gn, Go, mask=None):
    t_min=.5
    t_max=.2
    with_l2 = True

    patch_size = 16
    sift = np.zeros((128)) 

    if mask is not None:
        Gn = Gn * mask
    
    idx = 0
    for k in range(0, patch_size, 4):
        for l in range(0, patch_size, 4):
            hist = compute_histogram(Gn[l:l+4,k:k+4], Go[l:l+4,k:k+4])            
            sift[idx:idx+8] = hist
            idx += 8

    norm = np.linalg.norm(sift, ord=2)
    # min thresholding on norm
    if norm <= t_min:
        return np.zeros((128))
    # l2-normalization
    if with_l2:
        sift = sift / norm
    # max thresholding on values
    sift[sift >= t_max] = t_max
    # l2-normalization
    if with_l2:
        norm = np.linalg.norm(sift, ord=2)
        sift = sift / norm
    return sift

In [18]:
def compute_sift_image(I):
    x, y = dense_sampling(I)
    im = auto_padding(I)
    m = gaussian_mask()
    
    # Here, compute on the global image (norm, gradients)
    Gn, Go = compute_grad_mod_ori(I)
    
    sifts = np.zeros((len(x), len(y), 128))
    for i, xi in enumerate(x):
        for j, yj in enumerate(y):
            if xi + 16 <= Gn.shape[0] and yj + 16 <= Gn.shape[1]:  # it was usefull afterall
                sifts[i, j, :] = compute_sift_region(Gn[xi:xi+16, yj:yj+16], Go[xi:xi+16, yj:yj+16], m) # TODO SIFT du patch de coordonnee (xi, yj)
    return sifts

In [19]:
from numpy import asarray
from numpy import savez_compressed

In [20]:
def ComputeSiftDataset(inames):

    descriptors = []
    for x in inames:
        p = os.path.join(path, x)
        print(p)
        img = cv.imread(p)
        gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
        #print(gray.shape)
        des = compute_sift_image(gray)

        descriptors.append(des)
    
    return descriptors

In [21]:
descriptors = ComputeSiftDataset(inames)

images-photographes\daido-moriyama\16_daido_moriyama_24.jpg
images-photographes\daido-moriyama\16_daido_moriyama_25.jpg
images-photographes\daido-moriyama\16_daido_moriyama_26.jpg
images-photographes\daido-moriyama\16_daido_moriyama_27.jpg
images-photographes\daido-moriyama\21_moriyama_mep_050.jpg
images-photographes\daido-moriyama\21_moriyama_mep_051.jpg
images-photographes\daido-moriyama\21_moriyama_mep_059.jpg
images-photographes\daido-moriyama\21_moriyama_mep_063.jpg
images-photographes\daido-moriyama\21_moriyama_mep_064.jpg
images-photographes\daido-moriyama\21_moriyama_mep_065.jpg
images-photographes\daido-moriyama\21_moriyama_mep_066.jpg
images-photographes\daido-moriyama\21_moriyama_mep_068.jpg
images-photographes\daido-moriyama\21_moriyama_mep_073.jpg
images-photographes\daido-moriyama\21_moriyama_mep_075.jpg
images-photographes\daido-moriyama\21_moriyama_mep_077.jpg
images-photographes\daido-moriyama\21_moriyama_mep_078.jpg
images-photographes\daido-moriyama\21_moriyama_mep_0

images-photographes\irving-penn\17_irving_penn_19.jpg
images-photographes\irving-penn\17_irving_penn_21.jpg
images-photographes\irving-penn\17_irving_penn_23.jpg
images-photographes\irving-penn\17_irving_penn_29.jpg
images-photographes\irving-penn\17_irving_penn_31.jpg
images-photographes\irving-penn\17_irving_penn_33.jpg
images-photographes\irving-penn\17_irving_penn_35.jpg
images-photographes\irving-penn\17_irving_penn_39.jpg
images-photographes\irving-penn\17_irving_penn_41.jpg
images-photographes\irving-penn\17_irving_penn_43.jpg
images-photographes\irving-penn\17_irving_penn_45.jpg
images-photographes\irving-penn\17_irving_penn_47.jpg
images-photographes\irving-penn\17_irving_penn_49.jpg
images-photographes\irving-penn\17_irving_penn_51.jpg
images-photographes\irving-penn\17_irving_penn_53.jpg
images-photographes\irving-penn\17_irving_penn_55.jpg
images-photographes\irving-penn\17_irving_penn_57.jpg
images-photographes\irving-penn\17_irving_penn_59.jpg
images-photographes\irving-p

NameError: name 'dataset_feats' is not defined

In [ ]:
descriptors

In [23]:
len(descriptors)

280

In [24]:
descriptors[0]

array([[[0.0031295 , 0.        , 0.        , ..., 0.00115642,
         0.01750957, 0.00037196],
        [0.        , 0.        , 0.        , ..., 0.01601001,
         0.01861171, 0.01534671],
        [0.0114351 , 0.0094533 , 0.        , ..., 0.06153798,
         0.17042908, 0.03427139],
        ...,
        [0.02089614, 0.05787352, 0.07549245, ..., 0.06578476,
         0.03762011, 0.09189296],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.01242886, 0.        , 0.        , ..., 0.        ,
         0.02548922, 0.00165243],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.02480308, 0.10096261, ..., 0.01543152,
         0.        , 0.01920484],
        ...,
        [0.06707114, 0.09205   , 0.2075405 , ..., 0.        ,
         0.02663336, 0.11454061],
        [0. 

In [28]:
descriptors[0].shape

(425, 319, 128)

In [49]:
data = np.array(descriptors, dtype = object)


In [37]:
data[1].shape

(401, 301, 128)

In [50]:
savez_compressed('SIFTS DATA.npz', data) 

OSError: [Errno 28] No space left on device

In [31]:
savez_compressed('siftsV3.ndy', np.array(descriptors)) 

C:\Users\94\AppData\Local\Temp/ipykernel_14360/4190054146.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  savez_compressed('siftsV3.npz', np.array(descriptors))


KeyboardInterrupt: 

In [30]:
for d in descriptors:
    print(d.shape)

(425, 319, 128)
(401, 301, 128)
(377, 283, 128)
(387, 291, 128)
(225, 151, 128)
(151, 225, 128)
(146, 225, 128)
(225, 158, 128)
(225, 151, 128)
(225, 150, 128)
(170, 225, 128)
(225, 169, 128)
(225, 146, 128)
(225, 144, 128)
(152, 225, 128)
(151, 225, 128)
(151, 225, 128)
(149, 225, 128)
(150, 225, 128)
(75, 50, 128)
(225, 152, 128)
(225, 152, 128)
(156, 225, 128)
(161, 225, 128)
(52, 75, 128)
(225, 153, 128)
(144, 225, 128)
(149, 225, 128)
(150, 225, 128)
(153, 225, 128)
(225, 154, 128)
(225, 185, 128)
(225, 180, 128)
(180, 225, 128)
(174, 225, 128)
(166, 225, 128)
(220, 225, 128)
(225, 160, 128)
(171, 225, 128)
(225, 222, 128)
(171, 225, 128)
(225, 176, 128)
(225, 225, 128)
(225, 214, 128)
(172, 225, 128)
(225, 174, 128)
(225, 174, 128)
(220, 225, 128)
(225, 222, 128)
(225, 209, 128)
(225, 175, 128)
(225, 162, 128)
(225, 179, 128)
(225, 196, 128)
(187, 225, 128)
(225, 138, 128)
(225, 183, 128)
(225, 175, 128)
(225, 167, 128)
(225, 162, 128)
(225, 180, 128)
(225, 167, 128)
(174, 225, 1

In [48]:
data = np.load('siftsV2.npz')
data = np.array(data)

print(data[0].shape)

()
